In [1]:
import pandas as pd

#read csv files
df_f=pd.read_csv("follows.csv")
df_i=pd.read_csv("interests.csv")

df_f.columns = df_f.columns.str.strip()
df_i.columns = df_i.columns.str.strip()

#one-hot coding
df_i = pd.get_dummies(df_i,columns=['category'])
df_i = df_i.groupby('user_id').sum()

df_f = pd.get_dummies(df_f,columns=['followee_id'])
df_f = df_f.groupby('follower_id').sum()
df_f.index.names = ['user_id']

In [2]:
#rename df_f columns
df_f_cols= [df_f.columns.tolist()[i].strip('followee_id_') for i in range(len(df_f.columns.tolist()))]
df_f.columns= df_f_cols

In [3]:
#inner join two make both dataframes have same user_ids
df_test1 = pd.DataFrame.join(df_i,df_f,on='user_id',how='inner')
df_test1.shape

(2237, 4478)

In [4]:
#transpose datasets
df_reduced_i = df_test1.T.iloc[:883].T
df_reduced_f = df_test1.T.iloc[883:].T

In [5]:
#transpose datasets
df_f_t = df_reduced_f.T
df_f_t.index.names=['followee_id']
df_f_t.index=df_f_t.index.astype('int64')
df_reduced_i.index.names = ['followee_id']

In [6]:
#make followees are same as user_ids
df_test2 = pd.DataFrame.merge(df_reduced_i,df_f_t,left_index=True, right_index=True)
df_test2.index.names=['user_id']
df_test2.shape

(2237, 3120)

In [7]:
#transpose back to origin dataframes
df_interest = df_test2.T.iloc[:883].T
df_followee = df_test2.T.iloc[883:].T

In [58]:
#popular method
followee_id = df_followee.columns.values
total_follower = []

#calculate number of followers of each id
for n in range(0, 2237):
    tfollower = df_followee.iloc[:, n].sum()
    total_follower.append(tfollower)

#create a dataframe to list total followers of each user id and sort it by followers
data_top = {'user_id': followee_id, 'total_follower': total_follower}
df_top = pd.DataFrame(data=data_top)
df_top_sort = df_top.sort_values(by=['total_follower'], axis=0, ascending=False)


In [92]:
#create predicition matrix for recommended followees
df_recom_top = pd.DataFrame(columns =df_followee.columns.tolist(), index =df_followee.index.tolist())

In [93]:
#generate the recommendation followees for each id
for m in range(0, 2237):
    nrecommendation = df_followee.iloc[m].sum()
    recom_top_id = df_top_sort.user_id.values.tolist()[:nrecommendation]
    df_recom_top.at[followee_id[m], recom_top_id] = 1


In [74]:
from sklearn.metrics import precision_score
from statistics import mean

#precision score function
def prec_score(df_true, df_prediction): 
    pre = [precision_score(df_true.loc[i],df_prediction.loc[i])for i in df_prediction.index.values ]
    return mean(pre)

In [75]:
from sklearn.metrics import f1_score

#f1 score function
def fone_score(df_true, df_prediction):
    f1 = [f1_score(df_true.loc[i],df_prediction.loc[i]) for i in df_prediction.index.values]
    return mean(f1)

In [94]:
#prediction/ recommended followee
df_pred = df_recom_top.fillna(0)
df_true = df_followee

In [95]:
print("precision_score is: ",prec_score(df_true,df_pred))
print("f1_score is: ",fone_score(df_true,df_pred))

precision_score is:  0.7040866904574694
f1_score is:  0.7040866904574694
